# Playback OpenSim Mot file
In this notebook is shown how to load a mot file generate by OpenSim and play it back on the MyoSkeleton

In [1]:
import os
import time
from urllib.parse import urlparse

import mujoco
import numpy as np
import pandas as pd
import requests
import skvideo.io

def read_mot(filename):
    # --- Save the file ---
    if os.path.exists(filename):
        print(f"File '{filename}' already exists.")
    else:
        try:
            req = requests.get(url, allow_redirects=True, stream=True)
            req.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

            parsed_url = urlparse(url)
            filename = os.path.basename(parsed_url.path)
            if not filename:  # Handle cases where the path doesn't include a filename
                filename = "downloaded_file"  # Provide a default name
            print(f"File '{filename}' downloaded successfully.")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading file: {e}")
    # Initialize a counter for the number of rows to skip
    skiprows = 0

    # Open the file and read line by line
    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            # Increment the skiprows counter for each line until "endheader" is found
            if line.strip() == "endheader":
                break
            skiprows += 1

    # Now read the CSV file, skipping the determined number of rows
    df = pd.read_csv(filename, sep="\t", skiprows=skiprows + 1)

    return df



In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [44]:
url = "https://raw.githubusercontent.com/opensim-org/opensim-models/refs/heads/master/Pipelines/Gait2392_Simbody/OutputReference/subject01_walk1_ik.mot"
df = read_mot(url)

mj_model = mujoco.MjModel.from_xml_path(
    "../../../myosuite/simhive/myo_model/myoskeleton/myoskeleton.xml"
)
mj_data = mujoco.MjData(mj_model)

joint_names = [mj_model.joint(jn).name for jn in range(mj_model.njnt)]
subc = [c for c in df.columns if c in joint_names]

print(
    f"Joints in the Mot files that are not present in the MJC model: {set(subc) - set(joint_names)}"
)

# ---- camera settings
camera = mujoco.MjvCamera()
camera.azimuth = 90
camera.distance = 3
camera.elevation = -45.0
camera.lookat = [0,0,.75]
options_ref = mujoco.MjvOption()
options_ref.flags[:] = 0
options_ref.geomgroup[1:] = 0
renderer_ref = mujoco.Renderer(mj_model)
renderer_ref.scene.flags[:] = 0
frames=[]
from tqdm import tqdm
for t in tqdm(range(len(df)), desc="Rendering frames"):
    for jn in subc:
        mjc_j_idx = mj_model.joint(joint_names.index(jn)).qposadr
        mj_data.qpos[mjc_j_idx] = np.deg2rad(df[jn].loc[t])
        if "knee_angle" in jn:  # knee joints have negative sign in myosuite
            mj_data.qpos[mjc_j_idx] *= -1

    mujoco.mj_forward(mj_model, mj_data)
    renderer_ref.update_scene(mj_data, camera=camera)#, scene_option=options_ref)
    frame = renderer_ref.render()
    frames.append(frame)

os.makedirs('videos', exist_ok = True)
output_name = 'videos/playback_mot.mp4'
skvideo.io.vwrite(output_name, np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})
# show in the notebook
show_video('videos/playback_mot.mp4')

File 'subject01_walk1_ik.mot' downloaded successfully.
Joints in the Mot files that are not present in the MJC model: set()


Rendering frames: 100%|██████████| 211/211 [00:00<00:00, 448.01it/s]
